In [1]:
# Install the required libraries
!pip install pymongo
!pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 670.0/670.0 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.7/307.7 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.6/320.6 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.7 MB/s eta 0:00:00


In [2]:
import pymongo
from pymongo import MongoClient
import json
import re
from openai import OpenAI
from bson import ObjectId
from datetime import datetime
from abc import ABC, abstractmethod

In [3]:
# Connect to your MongoDB Atlas cluster
# Replace the connection string with your own
client = MongoClient("mongodb+srv://<username>:<password>@cluster0.r8pymzo.mongodb.net/")

# Access the sample_mflix database
db = client.sample_mflix

In [4]:
class PipelineStage(ABC):
    @abstractmethod
    def execute(self, data, **kwargs):
        pass

In [5]:
class SchemaExtractionStage(PipelineStage):
    def execute(self, data, **kwargs):
        def get_collection_schema(collection_name):
            collection = db[collection_name]
            sample_document = collection.find_one()
            schema = {}
            for field, value in sample_document.items():
                field_type = type(value).__name__
                if isinstance(value, list):
                    schema[field] = {"type": "array"}
                else:
                    schema[field] = {"type": field_type}
            return schema

        def generate_schema():
            collections_schema = {}
            for collection_name in db.list_collection_names():
                collections_schema[collection_name] = {"name": collection_name, "description": f"Contains detailed information about {collection_name}."}
                collections_schema[collection_name]["fields"] = get_collection_schema(collection_name)
            return collections_schema

        schema = generate_schema()
        # print(schema)
        data['schema'] = schema

        for collection_name in db.list_collection_names():
            schema = {"name": collection_name, "description": f"Contains detailed information about {collection_name}."}
            schema["fields"] = get_collection_schema(collection_name)
            with open(f"{collection_name}.json", "w") as f:
                json.dump(schema, f, indent=4)
        # print("schema files generated")
        return data

In [6]:
class ExampleDataAdditionStage(PipelineStage):
    def execute(self, data, **kwargs):
        movies_collection = db.movies
        movie_document = movies_collection.find_one()

        def add_example_data(schema, document):
            for field in schema['fields']:
                if field in document:
                    schema['fields'][field]['example'] = document[field]
                else:
                    schema['fields'][field]['example'] = None
            return schema

        with open('movies.json', 'r') as file:
            movies_schema = json.load(file)

        updated_schema = add_example_data(movies_schema, movie_document)

        class JSONEncoder(json.JSONEncoder):
            def default(self, obj):
                if isinstance(obj, ObjectId):
                    return str(obj)
                if isinstance(obj, datetime):
                    return obj.isoformat()
                return json.JSONEncoder.default(self, obj)

        with open('movies.json', 'w') as file:
            json.dump(updated_schema, file, indent=4, cls=JSONEncoder)

        # print("movies.json updated")

        data['updated_movies_schema'] = updated_schema
        return data

In [7]:
class RelationshipsIndexesStage(PipelineStage):
    def execute(self, data, **kwargs):
        def identify_relationships_and_indexes(db):
            relationships = []
            indexes = []
            collections = db.list_collection_names()

            for collection_name in collections:
                collection = db[collection_name]
                sample_document = collection.find_one()
                if not sample_document:
                    continue

                for field in sample_document:
                    if field.endswith('_id'):
                        related_collection_name = field[:-3] + 's'
                        if related_collection_name in collections:
                            relationship = {
                                "from_collection": collection_name,
                                "from_field": field,
                                "to_collection": related_collection_name,
                                "to_field": "_id",
                                "description": f"Links {collection_name} to {related_collection_name} through {field}."
                            }
                            relationships.append(relationship)

                            index = {
                                "from_collection": collection_name,
                                "index": field
                            }
                            indexes.append(index)
            return relationships, indexes

        with open('comments.json', 'r') as file:
            comments_schema = json.load(file)

        relationships, indexes = identify_relationships_and_indexes(db)
        for relationship in relationships:
            if relationship['from_collection'] == 'comments':
                comments_schema["relationships"] = relationship
                break

        for index in indexes:
            if index['from_collection'] == 'comments':
                comments_schema["indexes"] = index["index"]
                break

        with open('comments.json', 'w') as file:
            json.dump(comments_schema, file, indent=4)

        # print("comments.json updated")

        data['comments_schema'] = comments_schema
        return data

In [50]:
class OpenAIDescriptionStage(PipelineStage):
    def execute(self, data, **kwargs):
        def generate_description(prompt):
            client = OpenAI(
                api_key = <OpenAI_API_KEY>,
              )

            chat_completion = client.chat.completions.create(
                model = "gpt-3.5-turbo",
                messages = [
                    {"role": "user",
                     "content": prompt
                     }
                ],
                max_tokens = 300
            )

            return chat_completion

        # Load JSON file
        with open('movies.json', 'r') as f:
            data = json.load(f)

        # Extract field names
        fields = data['fields']

        # Construct the schema description
        json_schema = json.dumps(fields)

        # Generate description for each field
        prompt_str = f"""{json_schema} is the schema of collections, fields in a mongodb database.
Description refers to what information the collection or field holds.
Give the description as
field name : description
"""

        description_generated = generate_description(prompt_str)
        # print(description_generated)

        description_string = str(description_generated)
        # print(description_string)

        # Use regular expression to find the content
        pattern = re.compile(r"content=\"(.*?)\", role='assistant'", re.DOTALL)
        match = pattern.search(description_string)

        if match:
            content = match.group(1)
        #     print(content)
        # else:
        #     print("No content found")

        description_list = content.split("\\n")
        # print(description_list)

        for item in description_list:
            field, description_value = item.split(" : ", 1)
            data["fields"][field]["description"] = description_value
            # print(field, description_value)

        with open('movies.json', 'w') as f:
            json.dump(data, f, indent=4)

        # print("Descriptions updated and written to movies.json file.")

        return {"description_generated": description_generated}

In [9]:
class Pipeline:
    def __init__(self):
        self.stages = []

    def add_stage(self, stage):
        self.stages.append(stage)

    def execute(self, initial_data={}, **kwargs):
        data = initial_data
        for stage in self.stages:
            output = stage.execute(data, **kwargs)
            if output is not None:
                data.update(output)
        return data

In [51]:
if __name__ == "__main__":
    pipeline = Pipeline()
    pipeline.add_stage(SchemaExtractionStage())
    pipeline.add_stage(RelationshipsIndexesStage())
    pipeline.add_stage(OpenAIDescriptionStage())
    pipeline.add_stage(ExampleDataAdditionStage())

    result = pipeline.execute()
    print("Pipeline result:", result)

Pipeline result: {'schema': {'sessions': {'name': 'sessions', 'description': 'Contains detailed information about sessions.', 'fields': {'_id': {'type': 'ObjectId'}, 'user_id': {'type': 'str'}, 'jwt': {'type': 'str'}}}, 'movies': {'name': 'movies', 'description': 'Contains detailed information about movies.', 'fields': {'_id': {'type': 'ObjectId'}, 'plot': {'type': 'str'}, 'genres': {'type': 'array'}, 'runtime': {'type': 'int'}, 'cast': {'type': 'array'}, 'num_mflix_comments': {'type': 'int'}, 'poster': {'type': 'str'}, 'title': {'type': 'str'}, 'fullplot': {'type': 'str'}, 'countries': {'type': 'array'}, 'released': {'type': 'datetime'}, 'directors': {'type': 'array'}, 'writers': {'type': 'array'}, 'awards': {'type': 'dict'}, 'lastupdated': {'type': 'str'}, 'year': {'type': 'int'}, 'imdb': {'type': 'dict'}, 'type': {'type': 'str'}, 'tomatoes': {'type': 'dict'}}}, 'embedded_movies': {'name': 'embedded_movies', 'description': 'Contains detailed information about embedded_movies.', 'fiel